## Comparing the computational cost of quadratic sum

We will evaluate the computational cost of the following quadratic sum.

$$
\mathbf{x}^T A \mathbf{y} = \sum_{i=1}^n\sum_{j=1}^m x_iA_{ij}x_j
$$

where $A \in \mathbb{R}^{m\times n}$. 

The possible options to compute this include:
 1. Implementing with pure R function with loops
 1. Implementing with Rcpp/C++ function with loops
 1. Implementing with matrix multiplication in R
 
First, we define an arbitrary values of $\mathbf{x},~A,~\mathbf{y}$.

In [ ]:
x0 <- rep(1,10000)
y0 <- rep(1,10000)
A0 <- matrix(1,10000,10000)

It is obvious that $\mathbf{x}_0^T A_0 \mathbf{y}_0 = 10^8$.

A version of R loop implementation is below.

In [ ]:
quadsum_r_loop <- function(x, A, y) {
    n <- nrow(A)
    m <- ncol(A)
    sum <- 0
    for(i in 1:n) {
        for(j in 1:m){
            sum = sum + x[i]*A[i,j]*y[j]
        }
    }
    return (sum)
}

Let's evaluate how long it takes.

In [ ]:
system.time( print( quadsum_r_loop(x0, A0, y0) ) )

Rcpp/C++ implementation of the same algorithm can be implemented using `sourceCpp()` or `cppFunction()`. We will use `cppFunction()` here.

In [ ]:
library(Rcpp)
cppFunction('double quadsum_c_loop(NumericVector x, NumericMatrix A, NumericVector y) {
    double sum = 0;
    int n = A.nrow(), m = A.ncol();
    for(int i=0; i < n; ++i) {
        for(int j=0; j < m; ++j) {
            sum += x[i] * A(i,j) * x[j];
        }
    }
    return sum;
}')

Run a similar command to compare the running time

In [ ]:
system.time( print( quadsum_c_loop(x0, A0, y0) ) )

In [ ]:
quadsum_mat <- function(x, A, y) {
    return( x %*% A %*% y )
}
system.time( print( quadsum_mat(x0, A0, y0) ) )

The C++ version appears slower than actual speed because of inefficient optimization during compiliation. When working on a command line, matrix operation and C++ loop should be only ~3x slower.